In [ ]:
pip install python-terrier
pip install translate
pip install transquest
pip install deep-translator
pip install translation-quality-estimator
pip install tqdm

Add IR database

In [1]:
import pyterrier as pt
from typing import Callable
import pandas as pd
from pathlib import Path
from deep_translator import GoogleTranslator
import torch
from transquest.algo.sentence_level.monotransquest.run_model import MonoTransQuestModel
from tqe import TQE
from tqdm import tqdm

if not pt.started():
    pt.init()
de_database = pt.datasets.get_dataset("irds:mmarco/de/dev/small")
es_database = pt.datasets.get_dataset("irds:mmarco/es/dev/small")
fr_database = pt.datasets.get_dataset("irds:mmarco/fr/dev/small")
id_database = pt.datasets.get_dataset("irds:mmarco/id/dev/small")
pt_database = pt.datasets.get_dataset("irds:mmarco/pt/dev/small")

de_queries = de_database.get_topics()["query"].tolist()
print(de_database.get_topics()["query"])

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



          qid                                 query
0     1048585            was ist paula deens bruder
1           2           androgenrezeptor definieren
2      524332       behandlung von ohne medikamente
3     1048642                   was ist paranoid sc
4      524447  behandlung von krampfadern in beinen
...       ...                                   ...
6975   734979                   was ist koronar cta
6976   524166          transport zu uns bankstadion
6977   968921           woher stammen die nachnamen
6978   786375        was ist pr operative clearance
6979  1048565        wer spielt sebastian michaelis

[6980 rows x 2 columns]
          qid                                              query
0     1048585                     qu es el hermano de paula deen
1           2                   el receptor de andr genos define
2      524332  tratando dolores de cabeza por tensi n sin med...
3     1048642                             lo que es paranoico sc
4      524447  tratamiento

In [2]:
IDX_PATH = Path("index").absolute()

def evaluate(df: pd.DataFrame, dataset, rewrite_func: Callable[[str], str] = None) -> float:
    indexer = pt.IterDictIndexer(str(IDX_PATH))
    indexref = indexer.index(dataset.get_corpus_iter())

    bm25 = pt.BatchRetrieve(indexref, wmodel="BM25")
    tfidf = pt.BatchRetrieve(indexref, wmodel="TF_IDF")
    return pt.Experiment(
        [tfidf, bm25],
        df,
        dataset.get_qrels(),
        eval_metrics=["map"],
    )["map", "recip_rank"]

mmarco/es/dev/small documents:   1%|          | 77955/8841823 [00:26<47:01, 3106.27it/s]  

18:20:32.505 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (78021) - further warnings are suppressed


mmarco/es/dev/small documents: 100%|██████████| 8841823/8841823 [53:02<00:00, 2777.84it/s]  


19:15:01.616 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 117 empty documents


In [3]:
# Get translated queries and score
translated_de_to_es_queries = []

try:
    with open("translated_de_to_es_queries.txt", "r") as f:
        for line in f:
            translated_de_to_es_queries.append(line.strip())
except Exception as e:
    translator = GoogleTranslator(source="de", target="es")
    for de_query in tqdm(de_queries):
        translated_de_to_es_queries.append(translator.translate(de_query))

with open("translated_de_to_es_queries.txt", "w") as f:
    for q in translated_de_to_es_queries:
        f.write(q + "\n")

model = TQE()
translate_score = model.fit(de_queries, translated_de_to_es_queries)
print(sum(translate_score) / len(translate_score))

Computing embeddings for strings in list 1


Batches:   0%|          | 0/219 [00:00<?, ?it/s]

Computing embeddings for strings in list 2


Batches:   0%|          | 0/219 [00:00<?, ?it/s]

Computing cosine similarity scores
0.8054627507163317


In [5]:
# Evaluate querying score
test_topics = de_database.get_topics()
test_topics["query"] = translated_de_to_es_queries
print(evaluate(test_topics, es_database))

          qid                                           query
0     1048585              ¿Cuál es el hermano de Paula Deen?
1           2                  Definir receptor de andrógenos
2      524332                      tratamiento sin medicación
3     1048642                           ¿Qué es SC paranoico?
4      524447      Tratamiento de las varices en las piernas.
...       ...                                             ...
6975   734979                       ¿Qué es el cta coronario?
6976   524166  transporte al estadio del banco estadounidense
6977   968921                 ¿De dónde vienen los apellidos?
6978   786375               ¿Qué es el permiso preoperatorio?
6979  1048565          quien interpreta a sebastián michaelis

[6980 rows x 2 columns]
19:50:39.682 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 2,2 GiB of memory would be r

JavaException: JVM exception occurred: Failed to process qid 1048585 '¿Cuál es el hermano de Paula Deen?' -- Lexical error at line 1, column 35.  Encountered: <EOF> after : "" org.terrier.querying.parser.QueryParserException